## Generate a master.kml file from all kml files in a folder

In [3]:
import os
import chardet

# Path to the folder containing KML files
folder_path = 'data/pothole-videos'
master_kml_path = os.path.join(folder_path, 'master.kml')

# Generate the Master KML content
with open(master_kml_path, 'w') as master_kml:
    master_kml.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    master_kml.write('<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:gx="http://www.google.com/kml/ext/2.2">\n')
    master_kml.write('  <Document>\n')
    master_kml.write('    <name>All Tracks</name>\n')
    master_kml.write('    <open>1</open>\n')
    master_kml.write('    <visibility>1</visibility>\n')

    # Loop through each KML file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.kml') and file_name != 'master.kml':
            kml_path = os.path.join(folder_path, file_name)
            
            with open(kml_path, 'rb') as f:
                raw_data = f.read(10000)  # Read a sample
                result = chardet.detect(raw_data)

            encoding = result['encoding']
            print(f"Detected encoding: {encoding}")

            # Read each KML file's content and embed it directly
            with open(kml_path, 'r', encoding=encoding) as kml_file:
                inside_document = False
                for line in kml_file:
                    # Skip XML header, <kml> tags, and outer <Document> tags
                    if line.strip().startswith('<?xml') or line.strip().startswith('<kml'):
                        continue
                    if '<Document>' in line:
                        inside_document = True
                        continue
                    if '</Document>' in line:
                        inside_document = False
                        continue
                    
                    # Only add content inside the original Document tags
                    if inside_document:
                        # Add <open> and <visibility> tags to relevant elements
                        if "<Placemark>" in line or "<Folder>" in line:
                            master_kml.write(line)
                            master_kml.write('      <open>1</open>\n')
                            master_kml.write('      <visibility>1</visibility>\n')
                        else:
                            master_kml.write(line)
    
    master_kml.write('  </Document>\n')
    master_kml.write('</kml>\n')

print("Master KML file created with enhanced visibility settings.")


Detected encoding: UTF-16
Detected encoding: UTF-16
Detected encoding: UTF-16
Detected encoding: UTF-16
Master KML file created with enhanced visibility settings.


## Extract GPS coordinates from a MP4 video

In [2]:
#  A function to get the GPS coordinates from a video
import os
video_path = 'data/2024-12-30Chico/GX010013.MP4'
def get_gps_coords(video_path):
    # exiftool -p kml.fmt -ee3 data/2024-12-30Chico/GX010013.MP4 > data/2024-12-30Chico/GX010013.MP4.kml
    cmd = f'exiftool -p kml.fmt -ee3 {video_path} > {video_path}.kml'
    os.system(cmd)
    coords = [] # list of coordinates
    with open(f'{video_path}.kml', 'r') as f:
        lines = f.readlines()

    for line in lines:
        if 'coordinates' in line:
            # sample: <coordinates>-121.7889676,39.7399915,0</coordinates>
            line = line.replace('<coordinates>', '').replace(',0</coordinates>', '')
            coords.append([float(x) for x in line.split(',')])

    return coords
coords=get_gps_coords(video_path) # get the GPS coordinates from the video exif data
print("Coords:", coords, " length of Coords: ", len(coords))

Coords: [[-121.8431, 39.7286]]  length of Coords:  1


# Split or combine video files

In [ ]:
#Splitting the video into smaller videos and keeping the GPS coordinates

# check the metadata of the video
# exiftool -ee -G3 "data/2024-12-30Chico/GX010013_0-3.MP4"

# Split the video into smaller segments
# ffmpeg -i "data/2024-12-30Chico/GX010013.MP4" -ss 0:0:0 -to 0:0:3 -copy_unknown -map_metadata 0 -c copy -map 0:u "data/2024-12-30Chico/GX010013_0-3.MP4"
# ffmpeg -i "GX011222.MP4" -ss 0:0:0 -to 0:0:5 -copy_unknown -map_metadata 0 -c copy -map 0:u "GX011222_0-5.MP4"

# for Windows
# ffmpeg -i "GX011222.MP4" -ss 0:0:0 -to 0:0:6 -copy_unknown -map_metadata 0 -c copy -map 0:0 -map 0:1 -map 0:3 "GX011222_0-6.MP4"

import os
import subprocess
video_path = 'data/2024-12-30Chico/GX010013.MP4'
coords = get_gps_coords(video_path)
split_dir = video_path.replace('.MP4', '')
os.makedirs(split_dir, exist_ok=True)
for i, coord in enumerate(coords):
    # Split the video into smaller segments
    start_time = i
    end_time = i + 1
    split_video_path = os.path.join(split_dir, f'split_{i}.MP4')
    cmd = f'ffmpeg -i {video_path} -ss {start_time} -to {end_time} -c copy {split_video_path}'
    subprocess.run(cmd, shell=True)
    # Get the GPS coordinates for each segment
    with open(f'{split_video_path}.kml', 'w') as f:
        f.write(f'<coordinates>{coord[0]},{coord[1]},0</coordinates>')  # Write the GPS coordinates to the KML file
print("Splitting the video into smaller videos and keeping the GPS coordinates done.")



In [ ]:
# Downsize a video file from 120 fps to 24 fps, preserve the meta data including GPS info:
# ffmpeg -i "GX011216_14-22.MP4" -vf "fps=12" -map_metadata 0 -movflags use_metadata_tags "GX011216_14-22_12fps.MP4"
